**After classifying the questions as in [my previous kernel](https://www.kaggle.com/ozoozo/kaggle-survey-classifying-the-questions), let's dig into the responses of those classified questions. In this kernel, we will explore the responses to the multiple column questions.**

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

import gc 

PATH = "/kaggle/input/kaggle-survey-2021/"

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib import pyplot as plt

%matplotlib inline

In [ ]:
df = pd.read_csv(PATH + "kaggle_survey_2021_responses.csv")
df

In [ ]:
df.columns = [(col + ": " + df.loc[0,col]) for col in df.columns]
df = df.iloc[1:]
df

In [ ]:
all_count = len(df)
all_count

In [ ]:
cols = df.columns[df.columns.str.contains("Part_")]
multiple_column_questions=set()
for col in cols:
    c = col.split(":")[1].split("?")[0]
    multiple_column_questions.add(c)

n = 1
for question in multiple_column_questions:
    print(n, ":", question)
    n += 1

In [ ]:
def get_data(question):
    
    cols=[]
    counts=[]
    dictionary ={}

    n=-1
    for col in df.columns[df.columns.str.contains(question)]:
        n+=1
        if n==0:
            a = df[col].notna()
        else:
            a = a | df[col].notna()
        
    all_count = len(a[a==True])
    
    for col in df.columns[df.columns.str.contains(question)]:
        count = len(df[df[col].notna()])
        coll = " ".join(col.split("- ")[2].split())
        #print(count,coll)
        cols.append(col.split("-")[2])
        counts.append(count)
        dictionary[coll] = np.round(100 * count/all_count,2)

    dictionary = dict(sorted(dictionary.items(), key = lambda x: x[1], reverse = True))

    #for d in dictionary.keys():
    #    print(dictionary[d], "\t", d)

    return dictionary, all_count

In [ ]:
def plot_figure(dictionary, title, bar_color, annotation_color, title_color, y_tick_color):
    
    y_values = dictionary.keys()
    """
    y_values = []
    for y in dictionary.keys():
        a = y.split("(")
        if len(a)>1:
            a = a[0] +"\n("+ a[1]
            y_values.append(a)
        else:
            y_values.append(y)
    """
            
    a = list(dictionary.keys())
    try:
        none_rank = a.index("No / None")
    except:
        try:
            none_rank = a.index("None")
        except:
            try:
                none_rank = a.index("No")
            except:
                try:
                    none_rank = a.index("I do not share my work publicly")
                except:
                    none_rank = a.index("None of these activities are an important part of my role at work")

                

    other_rank =  a.index("Other")
    
    
    fig, ax = plt.subplots(1,1,figsize=(20,int(0.6*len(dictionary))))

    none_color = "#FF5555"
    other_color = "#229999"
    barcolor=[]
    for c in range(len(dictionary)):
        if c==none_rank:
            barcolor.append(none_color)
        elif c==other_rank:
            barcolor.append(other_color)            
        else:
            barcolor.append(bar_color)


    plt.barh(width=list(dictionary.values()), y=list(y_values), height=0.75,color = barcolor)

    a = -1
    for d in dictionary:
        a += 1
        if a == none_rank:
            plt.text(s = str(str(dictionary[d]) + "%"), x= dictionary[d], y=d ,font = 'Comic Sans MS', va='center',ha='left',color=none_color,fontsize=14)
        elif a==other_rank:
            plt.text(s = str(str(dictionary[d]) + "%"), x= dictionary[d], y=d ,font = 'Comic Sans MS', va='center',ha='left',color=other_color,fontsize=14)            
        else:
            plt.text(s = str(str(dictionary[d]) + "%"), x= dictionary[d], y=d ,font = 'Comic Sans MS', va='center',ha='left',color=annotation_color,fontsize=14)



    plt.title(title,font='Comic Sans MS', fontsize=20, color=title_color)
    plt.gca().invert_yaxis()
    ax.tick_params(axis='y', colors=y_tick_color,labelsize=14)
    ax.set_xlim([0, max(dictionary.values())+10])
    plt.gca().get_yticklabels()[none_rank].set_color(none_color) 
    plt.gca().get_yticklabels()[other_rank].set_color(other_color) 
    ax.get_xaxis().set_visible(False)
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    ax.spines['bottom'].set_visible(False)
    ax.spines['left'].set_visible(False)

    plt.show()

There is no response to the question of "In the next 2 years, do you hope to become more familiar with any of these specific data storage products":

In [ ]:
df[df.columns[df.columns.str.contains("In the next 2 years, do you hope to become more familiar with any of these specific data storage products")]].sum()

In [ ]:
multiple_column_questions.remove(" In the next 2 years, do you hope to become more familiar with any of these specific data storage products")

Let's have a look at the responses to the multiple-column-questions by using graphs:

In [ ]:
for question in list(multiple_column_questions):
    dictionary, all_count = get_data(question.replace("(","\(").replace(")","\)"))
    plot_figure(dictionary,question+", total responses: "+str(all_count),"#FF925D","#19718C","#19718C","#19718C")

I hope, the above work can facilitate your diving into deeper analysis of the responses of this survey.